In [91]:
import requests
from bs4 import BeautifulSoup

result = requests.get("thttps://www.w3schools.com/html/tryit.asp?filename=tryhtml_lists_intro")
src = result.content

soup = BeautifulSoup(src, 'lxml')

In [76]:
lists = soup.find("ul")

In [77]:
print(lists)

<ul>
  <li>Coffee</li>
  <li>Tea</li>
  <li>Milk</li>
</ul>


In [85]:
items = lists.find_all("li")
items

[<li>Coffee</li>, <li>Tea</li>, <li>Milk</li>]

In [90]:
for item in items:
    print(item.get_text())

Coffee
Tea
Milk


In [30]:
soup.find?

In [39]:
rows[0]

<tr>
<th data-sort-type="number">Rank</th>
<th>Country<br/>(or dependent territory)</th>
<th>Population</th>
<th>% of world</th>
<th>Date</th>
<th class="unsortable">Source<br/>(official or UN)
</th></tr>

# Downloading a table from Wikipedia

We aim to get a list of countries sorted by their population size:
https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population

In [145]:
import requests
from bs4 import BeautifulSoup
import re
import dateutil

result = requests.get("https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population")
src = result.content


soup = BeautifulSoup(src, 'lxml')

In [146]:
r = soup.find_all("table")

In [147]:
# Make sure that the table contains the Rank column
assert "Rank" in str(r[0])

In [148]:
table = r[0]

In [165]:
rows = soup.find("table").find("tbody").find_all("tr")

countries = []

for row in rows[1:-1]:
    cells = row.find_all(["td", "th"])
    
    rank = cells[0].get_text().strip()  # remove any whitespaces before and after the rank
    
    country = cells[1].get_text().strip()
    country_clean = re.findall(r'[\w\s()]+', country)[0]
    
    size = cells[2].get_text()
    size_clean = int(re.sub(',', '', size))
    
    percentage = cells[3].get_text()
    percentage = float(re.findall(r'[\d\.]+', percentage)[0])
    
    date = cells[4].get_text()
    date = dateutil.parser.parse('2018-06-29 22:21:41')
    
    
    source = cells[5].get_text().strip()
    source_ref = int(re.findall(r'\[(\d+)\]', source)[0])
    
    countries.append([rank, 
                      country_clean, 
                      size_clean, 
                      percentage, 
                      #date, 
                      source_ref])
    
for c in countries:
    print(c)

['1', 'China', 1404832520, 18.0, 3]
['2', 'India', 1368306995, 17.5, 4]
['3', 'United States', 330459799, 4.23, 5]
['4', 'Indonesia', 269603400, 3.45, 6]
['5', 'Pakistan', 220892331, 2.82, 2]
['6', 'Brazil', 212188422, 2.71, 7]
['7', 'Nigeria', 206139587, 2.64, 2]
['8', 'Bangladesh', 169457738, 2.17, 8]
['9', 'Russia', 146748590, 1.88, 9]
['10', 'Mexico', 127792286, 1.63, 10]
['11', 'Japan', 125810000, 1.61, 11]
['12', 'Philippines', 109284207, 1.4, 12]
['13', 'DR Congo', 101935800, 1.3, 13]
['14', 'Egypt', 101036828, 1.29, 14]
['15', 'Ethiopia', 100829000, 1.29, 15]
['16', 'Vietnam', 96484000, 1.23, 16]
['17', 'Iran', 83857606, 1.07, 17]
['18', 'Turkey', 83154997, 1.06, 18]
['19', 'Germany', 83122889, 1.06, 19]
['20', 'France', 67132000, 0.859, 20]
['21', 'United Kingdom', 66796807, 0.854, 21]
['22', 'Thailand', 66563392, 0.851, 22]
['23', 'Italy', 60062012, 0.768, 23]
['24', 'South Africa', 59622350, 0.762, 24]
['25', 'Tanzania', 57637628, 0.737, 25]
['26', 'Myanmar', 54817919, 0.701

**Attention**: Beautiful Soup does not execute Javascript. This means that you the code in the Google Chrome inspector might look different to the original source code. 

In [30]:
soup.find?